# ItaData2024
propositional example

In [1]:
using Pkg
Pkg.activate(".")
using MLJ, ModalDecisionTrees
using SoleDecisionTreeInterface, Sole, SoleData
using CategoricalArrays
using DataFrames, JLD2, CSV
using Audio911
using Random
using StatsBase, Catch22
using Test
using Plots

  Activating project at `~/Documents/Aclai/audio-rules2024`


### Open .jld2 file
the file contains 504 samples of respiratory sound, labeled with 2 classes: healty and pneumonia

In [2]:
ds_path = "/datasets/respiratory_Healthy_Pneumonia"
# ds_path = "/datasets/respiratory_Healthy_Bronchiectasis"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

### Audio features extraction function
This function is called for every audio sample and extract 51 features:
26 bands of the mel spectrogram,
13 coefficients of the mfcc
12 spectral features: centroid, crest, entropy, f0, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [3]:
sr = 8000
audioparams = (
    sr = sr,
    nfft = 256,
    nbands = 14,
    freq_range = (300, round(Int, sr / 2)),
    db_scale = true,
)

function afe(x::AbstractVector{Float64}; sr::Int64, nfft::Int64, nbands::Int64, freq_range::Tuple{Int64, Int64}, db_scale::Bool, get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = sr
    norm = true
    speech_detection = false
    # stft module
    nfft = nfft
    win_type = (:hann, :periodic)
    win_length = nfft
    overlap_length = round(Int, nfft / 2)
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = nbands
    scale = :mel_htk                        # :mel_htk, :mel_slaney, :erb, :bark
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = freq_range
    # mel spectrogram module
    db_scale = db_scale

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        nfft=nfft,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.data.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    hcat(
        melspec.spec',
        spect.centroid,
        spect.crest,
        spect.entropy,
        spect.flatness,
        spect.flux,
        spect.kurtosis,
        spect.rolloff,
        spect.skewness,
        spect.decrease,
        spect.slope,
        spect.spread
    );
end

afe (generic function with 1 method)

### Compute DataFrame of features

In [4]:
color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36)
freq = round.(Int, afe(x[1, :audio]; audioparams..., get_only_melfreq=true))
r_select = r"\e\[\d+m(.*?)\e\[0m"

catch9_f = ["max", "min", "mean", "med", "std", "bsm", "bsd", "qnt", "3ac"]
variable_names = vcat([
    vcat(
        ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz->$j\e[0m" for i in 1:audioparams.nbands]...,
        "\e[$(color_code[:cyan])mcntrd->$j\e[0m", "\e[$(color_code[:cyan])mcrest->$j\e[0m",
        "\e[$(color_code[:cyan])mentrp->$j\e[0m", "\e[$(color_code[:cyan])mflatn->$j\e[0m", "\e[$(color_code[:cyan])mflux->$j\e[0m",
        "\e[$(color_code[:cyan])mkurts->$j\e[0m", "\e[$(color_code[:cyan])mrllff->$j\e[0m", "\e[$(color_code[:cyan])mskwns->$j\e[0m",
        "\e[$(color_code[:cyan])mdecrs->$j\e[0m", "\e[$(color_code[:cyan])mslope->$j\e[0m", "\e[$(color_code[:cyan])msprd->$j\e[0m"
    )
    for j in catch9_f
]...)

# cell 5 - Data compression for propositional analysis
catch9 = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    std,
    Catch22.SB_BinaryStats_mean_longstretch1,
    Catch22.SB_BinaryStats_diff_longstretch0,
    Catch22.SB_MotifThree_quantile_hh,
    Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]

X = DataFrame([name => Float64[] for name in [match(r_select, v)[1] for v in variable_names]])

audio_feats = [afe(row[:audio]; audioparams...) for row in eachrow(x)]
push!(X, vcat([vcat([map(func, eachcol(row)) for func in catch9]...) for row in audio_feats])...)

yc = CategoricalArray(y);

### Compute train and test sets

In [5]:
train_ratio = 0.8

train, test = partition(eachindex(yc), train_ratio, shuffle=true)
X_train, y_train = X[train, :], yc[train]
X_test, y_test = X[test, :], yc[test]

println("Training set size: ", size(X_train), " - ", length(y_train))
println("Test set size: ", size(X_test), " - ", length(y_test))

Training set size: (403, 225) - 403
Test set size: (101, 225) - 101


### Train a model

In [6]:
learned_dt_tree = begin
    Tree = MLJ.@load DecisionTreeClassifier pkg=DecisionTree
    model = Tree(max_depth=-1, )
    mach = machine(model, X_train, y_train)
    fit!(mach)
    fitted_params(mach).tree
end

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/paso/.julia/packages/MLJModels/8W54X/src/loading.jl:159


import MLJDecisionTreeInterface ✔


┌ Info: Training machine(DecisionTreeClassifier(max_depth = -1, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


cntrd->std < 185.4
├─ mel8=1583Hz->std < 0.3685
│  ├─ kurts->min < 3.279
│  │  ├─ mel8=1583Hz->std < 0.1972
│  │  │  ├─ Pneumonia (9/9)
│  │  │  └─ mel9=1831Hz->qnt < 1.986
│  │  │     ├─ Pneumonia (2/2)
│  │  │     └─ Healthy (9/9)
│  │  └─ mel7=1359Hz->mean < -4.432
│  │     ├─ entrp->bsm < 5.5
│  │     │  ├─ mel5=975Hz->med < -5.615
│  │     │  │  ⋮
│  │     │  │  
│  │     │  └─ Pneumonia (123/123)
│  │     └─ Healthy (1/1)
│  └─ mel14=3539Hz->max < -6.269
│     ├─ mel3=663Hz->std < 1.182
│     │  ├─ Healthy (29/29)
│     │  └─ Pneumonia (2/2)
│     └─ mel8=1583Hz->std < 0.6294
│        ├─ mel2=529Hz->min < -3.655
│        │  ├─ Pneumonia (32/32)
│        │  └─ Healthy (1/1)
│        └─ kurts->qnt < 2.13
│           ├─ Healthy (13/13)
│           └─ Pneumonia (5/5)
└─ decrs->med < -1.063
   ├─ Pneumonia (14/14)
   └─ mel8=1583Hz->std < 0.1904
      ├─ Pneumonia (5/5)
      └─ sprd->max < 790.2
         ├─ slope->mean < -1.027e-6
         │  ├─ mel5=975Hz->mean < -4.548
         │  

### Model inspection & rule study

In [7]:
sole_dt = solemodel(learned_dt_tree)
# Make test instances flow into the model, so that test metrics can, then, be computed.
apply!(sole_dt, X_test, y_test);
# Print Sole model
printmodel(sole_dt; show_metrics = true, variable_names_map = variable_names);

▣ cntrd->std < 185.36450136158544
├✔ mel8=1583Hz->std < 0.36849027868320827
│├✔ kurts->min < 3.2789095834200026
││├✔ mel8=1583Hz->std < 0.19723589763729776
│││├✔ Pneumonia : (ninstances = 3, ncovered = 3, confidence = 1.0, lift = 1.0)
│││└✘ mel9=1831Hz->qnt < 1.986086285479107
│││ ├✔ Pneumonia : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
│││ └✘ Healthy : (ninstances = 3, ncovered = 3, confidence = 1.0, lift = 1.0)
││└✘ mel7=1359Hz->mean < -4.432173583301696
││ ├✔ entrp->bsm < 5.5
││ │├✔ mel5=975Hz->med < -5.61451455943313
││ ││├✔ Pneumonia : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
││ ││└✘ Healthy : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
││ │└✘ Pneumonia : (ninstances = 31, ncovered = 31, confidence = 0.94, lift = 1.0)
││ └✘ Healthy : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
│└✘ mel14=3539Hz->max < -6.269434032853102
│ ├✔ mel3=663Hz->std < 1.1824394187533702
│ │├✔ Healthy : (ninstances = 3, ncovered = 3, co

### Extract rules that are at least as good as a random baseline model

In [8]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map = variable_names);

▣ (cntrd->std < 185.36450136158544) ∧ (mel8=1583Hz->std < 0.36849027868320827) ∧ (kurts->min < 3.2789095834200026) ∧ (mel8=1583Hz->std < 0.19723589763729776)  ↣  Pneumonia : (ninstances = 101, ncovered = 3, coverage = 0.03, confidence = 1.0, lift = 1.98, natoms = 4)
▣ (cntrd->std < 185.36450136158544) ∧ (mel8=1583Hz->std < 0.36849027868320827) ∧ (kurts->min < 3.2789095834200026) ∧ (¬(mel8=1583Hz->std < 0.19723589763729776)) ∧ (¬(mel9=1831Hz->qnt < 1.986086285479107))  ↣  Healthy : (ninstances = 101, ncovered = 3, coverage = 0.03, confidence = 1.0, lift = 2.02, natoms = 5)
▣ (cntrd->std < 185.36450136158544) ∧ (mel8=1583Hz->std < 0.36849027868320827) ∧ (¬(kurts->min < 3.2789095834200026)) ∧ (mel7=1359Hz->mean < -4.432173583301696) ∧ (¬(entrp->bsm < 5.5))  ↣  Pneumonia : (ninstances = 101, ncovered = 31, coverage = 0.31, confidence = 0.94, lift = 1.85, natoms = 5)
▣ (cntrd->std < 185.36450136158544) ∧ (¬(mel8=1583Hz->std < 0.36849027868320827)) ∧ (mel14=3539Hz->max < -6.269434032853102) 

### Simplify rules while extracting and prettify result

In [9]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map = variable_names);

▣ (cntrd->std < 185.36) ∧ (mel8=1583Hz->std < 0.2) ∧ (kurts->min < 3.28)  ↣  Pneumonia : (ninstances = 101, ncovered = 3, coverage = 0.03, confidence = 1.0, lift = 1.98, natoms = 3)
▣ (cntrd->std < 185.36) ∧ (mel8=1583Hz->std ∈ [0.2,0.37)) ∧ (kurts->min < 3.28) ∧ (mel9=1831Hz->qnt ≥ 1.99)  ↣  Healthy : (ninstances = 101, ncovered = 3, coverage = 0.03, confidence = 1.0, lift = 2.02, natoms = 4)
▣ (cntrd->std < 185.36) ∧ (mel8=1583Hz->std < 0.37) ∧ (kurts->min ≥ 3.28) ∧ (mel7=1359Hz->mean < -4.43) ∧ (entrp->bsm ≥ 5.5)  ↣  Pneumonia : (ninstances = 101, ncovered = 31, coverage = 0.31, confidence = 0.94, lift = 1.85, natoms = 5)
▣ (cntrd->std < 185.36) ∧ (mel8=1583Hz->std ≥ 0.37) ∧ (mel14=3539Hz->max < -6.27) ∧ (mel3=663Hz->std < 1.18)  ↣  Healthy : (ninstances = 101, ncovered = 3, coverage = 0.03, confidence = 0.67, lift = 1.35, natoms = 4)
▣ (cntrd->std < 185.36) ∧ (mel8=1583Hz->std ∈ [0.37,0.63)) ∧ (mel14=3539Hz->max ≥ -6.27) ∧ (mel2=529Hz->min < -3.65)  ↣  Pneumonia : (ninstances = 101

### Directly access rule metrics

In [10]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

13-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 101, ncovered = 3, coverage = 0.0297029702970297, confidence = 1.0, lift = 1.9803921568627452, natoms = 4)
 (ninstances = 101, ncovered = 3, coverage = 0.0297029702970297, confidence = 1.0, lift = 2.02, natoms = 5)
 (ninstances = 101, ncovered = 31, coverage = 0.3069306930693069, confidence = 0.9354838709677419, lift = 1.8526249209361163, natoms = 5)
 (ninstances = 101, ncovered = 3, coverage = 0.0297029702970297, confidence = 0.6666666666666666, lift = 1.3466666666666667, natoms = 4)
 (ninstances = 101, ncovered = 6, coverage = 0.0594059405940594, confidence = 0.8333333333333334, lift = 1.650326797385621, natoms = 5)
 (ninstances = 101, ncovered = 2, coverage = 0.019801980198019802, confidence = 1.0, lift = 2.02, natoms = 5)
 (ninstances = 101, ncovered = 3, coverage = 0.0297029702970297, confidence = 0.6666666666666666, lift = 1.3466

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [11]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map = variable_names);

▣ (cntrd->std < 185.36450136158544) ∧ (mel8=1583Hz->std ≥ 0.6294040098220748) ∧ (mel14=3539Hz->max ≥ -6.269434032853102) ∧ (kurts->qnt ≥ 2.13025203953131)  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.9803921568627452, natoms = 4, height = 3)
▣ (cntrd->std ≥ 185.36450136158544) ∧ (decrs->med ≥ -1.0634279559400643) ∧ (mel8=1583Hz->std ≥ 0.1903690018708441) ∧ (sprd->max < 790.2373947340018) ∧ (slope->mean ≥ -1.0265025776046623e-6)  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.9803921568627452, natoms = 5, height = 4)
▣ (cntrd->std ≥ 185.36450136158544) ∧ (decrs->med ≥ -1.0634279559400643) ∧ (mel8=1583Hz->std ≥ 0.1903690018708441) ∧ (sprd->max < 790.2373947340018) ∧ (slope->mean < -1.0265025776046623e-6) ∧ (mel5=975Hz->mean < -4.548205752283771) ∧ (mel9=1831Hz->mean < -5.611057754685882)  ↣  Healthy : (ninstances = 101, ncovered = 2, coverage = 0.019801980198019802

### Pretty table of rules and their metrics

In [12]:
metricstable(interesting_rules; variable_names_map = variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────┬──────────┬────────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                                                                                                       Antecedent │ Consequent │ ninstances │ ncovered │   coverage │ confidence │    lift │ natoms │ height │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────